# Wine Quality Classification

#### Group Members
<div style="display: inline">brandon.hemminghaus@ontariotechu.net</div> <br>

<div style="display: inline">saffana.ahammed@ontariotechu.net</div><br>

<div style="display: inline">tristan.ro@ontariotechu.net</div><br>



In [1]:
import torch
from torch import (nn, optim)
from torch.utils.data import (Dataset, DataLoader, random_split, TensorDataset)
import torchtext
import pandas as pd
import numpy as np
import warnings
from torch.optim import (Optimizer, Adam)
from torch.nn.functional import cross_entropy

ModuleNotFoundError: No module named 'torch'

## Data Import and Preprocessing

In [2]:
red = pd.read_csv('winequality-red.csv', sep = ';')
white = pd.read_csv('winequality-white.csv', sep = ';')

In [3]:
# merge
wine = pd.concat([red, white], ignore_index=True)

In [4]:
wine.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides   
0            7.4              0.70         0.00             1.9      0.076  \
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates   
0                 11.0                  34.0   0.9978  3.51       0.56  \
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5

In [5]:
len(wine)

6497

### Normalize features

In [6]:
wine.iloc[:, :-1] = (wine.iloc[:, :-1] - wine.iloc[:, :-1].min()) / (wine.iloc[:, :-1].max() - wine.iloc[:, :-1].min())

In [7]:
wine.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides   
0       0.297521          0.413333     0.000000        0.019939   0.111296  \
1       0.330579          0.533333     0.000000        0.030675   0.147841   
2       0.330579          0.453333     0.024096        0.026074   0.137874   
3       0.611570          0.133333     0.337349        0.019939   0.109635   
4       0.297521          0.413333     0.000000        0.019939   0.111296   

   free sulfur dioxide  total sulfur dioxide   density        pH  sulphates   
0             0.034722              0.064516  0.206092  0.612403   0.191011  \
1             0.083333              0.140553  0.186813  0.372093   0.258427   
2             0.048611              0.110599  0.190669  0.418605   0.241573   
3             0.055556              0.124424  0.209948  0.341085   0.202247   
4             0.034722              0.064516  0.206092  0.612403   0.191011   

    alcohol  quality  
0  0.202899        5  
1  0.260870        5  
2  0.260870        5  
3  0.260870        6  
4  0.202899        5

### Convert quality into buckets

The quality of wines (1-10) was converted to four categories for simplier classification

- 1-3 --> 0
- 4-5 --> 1
- 6-7 --> 2
- 8-10 --> 3

In [8]:
wine['quality'].unique()

array([5, 6, 7, 4, 8, 3, 9])

In [9]:
def convert_quality(label):
    if label <= 3:
        return 0
    elif label <= 5:
        return 1
    elif label <= 7:
        return 2
    else:
        return 3

In [10]:
wine['quality'] = wine['quality'].apply(convert_quality)

In [11]:
wine['quality'].unique()

array([1, 2, 3, 0])

- we now have 4 labels instead of 7

## Split into test and training data

In [12]:
# sepereate measurement data and class
x = wine.drop('quality', axis = 1)
y = wine['quality']
    
# convert to tensors
x_tensor = torch.tensor(x.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.int64)
    
# create dataset
dataset = TensorDataset(x_tensor, y_tensor)

# Define the sizes of the training and test sets
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset into training and test sets
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

## Model

This NN includes two BatchNorm1d layers which normalize the input to each layer. This can reduce the internal covariate shift. There is also a residual layer added between the 2nd and third layer and a softmax activation function for the output.

In [13]:
class NeuralNetClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bnorm1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bnorm2 = nn.BatchNorm1d(hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bnorm1(x)
        x = nn.functional.relu(x)
        residual = x # residual layer
        x = self.fc2(x)
        x = nn.functional.relu(x + residual) # add residual
        x = self.bnorm2(x)
        x = self.output(x)
        return nn.functional.softmax(x, dim=-1)


## Training Function

In [14]:
def train(model: NeuralNetClassifier,
          train_dataset: Dataset,
          learning_rate: float,
          epochs: int,
          max_batches=None) -> pd.DataFrame:
    
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    
    results = {'train_loss': [], 'train_acc': []}
    
    for epoch in range(epochs):
        losses = []
        
        
        for batch_idx, (x, target) in enumerate(train_data_loader):
            
            # conditon for batch size
            if max_batches is not None and batch_idx >= max_batches:
                break
                
            optimizer.zero_grad()
            y = model(x)
            loss = loss_fn(y, target)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            
        epoch_loss = np.mean(losses)
        results['train_loss'].append(epoch_loss)
        print("Epoch {}: loss={:.4f}".format(epoch, epoch_loss))
        
        # for traing accuracy
        with torch.no_grad():
            success = 0
            total = 0
            for x, target in train_data_loader:
                y = model(x)
                pred = y.argmax(axis=1)
                success += (pred == target).sum().item()
                total += target.shape[0]
            accuracy = success/total
            results['train_acc'].append(accuracy)
            print("Accuracy = {:.2f}".format(accuracy))

    return pd.DataFrame(results)

### Model Training

In [22]:
# parameters
input_size = 11
hidden_size = 64
output_size = 4

train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

neural_net_classifier = NeuralNetClassifier(input_size, hidden_size, output_size)

history_nn = train(neural_net_classifier, train_data_loader, 0.001, epochs = 30, max_batches = 100)

Epoch 0: loss=1.2188
Accuracy = 0.67
Epoch 1: loss=1.0733
Accuracy = 0.70
Epoch 2: loss=1.0432
Accuracy = 0.72
Epoch 3: loss=1.0316
Accuracy = 0.72
Epoch 4: loss=1.0253
Accuracy = 0.73
Epoch 5: loss=1.0167
Accuracy = 0.73
Epoch 6: loss=1.0184
Accuracy = 0.73
Epoch 7: loss=1.0115
Accuracy = 0.74
Epoch 8: loss=1.0109
Accuracy = 0.74
Epoch 9: loss=1.0039
Accuracy = 0.75
Epoch 10: loss=1.0020
Accuracy = 0.74
Epoch 11: loss=1.0034
Accuracy = 0.74
Epoch 12: loss=0.9949
Accuracy = 0.75
Epoch 13: loss=0.9969
Accuracy = 0.75
Epoch 14: loss=1.0038
Accuracy = 0.75
Epoch 15: loss=0.9939
Accuracy = 0.75
Epoch 16: loss=0.9855
Accuracy = 0.75
Epoch 17: loss=0.9864
Accuracy = 0.76
Epoch 18: loss=0.9868
Accuracy = 0.76
Epoch 19: loss=0.9913
Accuracy = 0.76
Epoch 20: loss=0.9847
Accuracy = 0.76
Epoch 21: loss=0.9833
Accuracy = 0.76
Epoch 22: loss=0.9862
Accuracy = 0.76
Epoch 23: loss=0.9782
Accuracy = 0.77
Epoch 24: loss=0.9789
Accuracy = 0.76
Epoch 25: loss=0.9840
Accuracy = 0.77
Epoch 26: loss=0.9737


### Model Testing

In [23]:
#test acc
def test_saved_model(model):

    loss = nn.CrossEntropyLoss()
    dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
    acc = 0
    with torch.no_grad():
        for xs, targets in dataloader:
            xs, targets = xs, targets
            ys = model(xs)
            acc += (ys.argmax(axis=1) == targets).sum().item()
    acc = acc / len(test_dataset) * 100
    print("Saved model has test accuracy = %.2f" % acc)


In [27]:
torch.save(neural_net_classifier, 'mymodel.pt')

In [32]:
model = torch.load('./mymodel.pt')
test_saved_model(model)

Saved model has test accuracy = 74.05
